## Step 1: Setting Up the Python Application

In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install plotly

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install --upgrade chromadb

Defaulting to user installation because normal site-packages is not writeable


In [5]:
pip install -U matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os

os.environ['OPENAI_API_KEY']=""

In [7]:
import openai
import pandas as pd
import csv
import chromadb
import torch
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from openai.embeddings_utils import get_embedding
from transformers import AutoTokenizer, AutoModel
from chromadb.utils import embedding_functions
from chromadb.config import Settings
from io import StringIO

## Step 2: Generating Real Estate Listings

In [8]:
model_name = 'gpt-3.5-turbo'
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature)
PROMPT_TEMPLATE = """
Generate a CSV file with ten real estate listings based on the sample listing format provided below. Structure the CSV with clear headers for each column: Neighborhood, Price, Bedrooms, Bathrooms, House Size, Description, and Neighborhood Description. Follow the example provided to format each subsequent row with information specific to a different property listing. Make sure you generate 10 unique listings.
{sample_listing}
"""
sample_listing="""
Neighborhood: Green Oaks
Price: "$800,000"
Bedrooms: 3
Bathrooms: 2
House Size: "2,000 sqft"

Description: "Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. Priced at $800,000, this charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem."

Neighborhood Description: "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
"""
prompt = PromptTemplate(
    input_variables=["sample_listing"],
    template=PROMPT_TEMPLATE
)

query = prompt.format(
    sample_listing=sample_listing,
)
print(query)


Generate a CSV file with ten real estate listings based on the sample listing format provided below. Structure the CSV with clear headers for each column: Neighborhood, Price, Bedrooms, Bathrooms, House Size, Description, and Neighborhood Description. Follow the example provided to format each subsequent row with information specific to a different property listing. Make sure you generate 10 unique listings.

Neighborhood: Green Oaks
Price: "$800,000"
Bedrooms: 3
Bathrooms: 2
House Size: "2,000 sqft"

Description: "Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. Priced at $800,000, this charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainab

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [9]:
response = llm(query)
print(response)

Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
Willow Creek,"$650,000",4,3,"2,500 sqft","Located in the peaceful neighborhood of Willow Creek, this spacious 4-bedroom, 3-bathroom home is perfect for a growing family. With a large backyard and modern finishes throughout, this home is ideal for entertaining guests or relaxing in comfort.","Willow Creek offers a serene setting with tree-lined streets and nearby parks. Enjoy the convenience of shopping centers and top-rated schools within walking distance."
Sunset Hills,"$1,200,000",5,4,"3,800 sqft","This luxurious 5-bedroom, 4-bathroom home in Sunset Hills features high-end finishes, a gourmet kitchen, and a private backyard oasis with a pool and spa. With stunning views of the sunset from the master bedroom balcony, this home is perfect for those who appreciate luxury living.","Sunset Hills is known for its upscale homes and scenic views. Enjoy easy access to hiking trails, golf courses, and fine di

In [10]:
data = StringIO(response)

# Define the filename for your CSV
filename = "real_estate_listings.csv"

# Write data to CSV file
with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Read and write the header row
    header = next(csv.reader(data))
    csvwriter.writerow(header)
    # Read and write the remaining rows
    csvwriter.writerows(csv.reader(data))

print(f"CSV file '{filename}' has been created successfully.")

CSV file 'real_estate_listings.csv' has been created successfully.


In [11]:
df = pd.read_csv(filename)

In [12]:
df.head(10)

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,Willow Creek,"$650,000",4,3.0,"2,500 sqft",Located in the peaceful neighborhood of Willow...,Willow Creek offers a serene setting with tree...
1,Sunset Hills,"$1,200,000",5,4.0,"3,800 sqft","This luxurious 5-bedroom, 4-bathroom home in S...",Sunset Hills is known for its upscale homes an...
2,Riverfront Estates,"$900,000",3,2.5,"2,300 sqft","Welcome to Riverfront Estates, where this 3-be...",Riverfront Estates is a waterfront community w...
3,Downtown Loft District,"$500,000",2,2.0,"1,200 sqft",Live in the heart of the city in this modern 2...,The Downtown Loft District is a vibrant neighb...
4,Mountain View Terrace,"$750,000",3,2.5,"2,100 sqft",Enjoy panoramic mountain views from this 3-bed...,Mountain View Terrace offers a peaceful settin...
5,Lakefront Retreat,"$1,500,000",4,3.5,"3,500 sqft","Escape to this stunning 4-bedroom, 3.5-bathroo...",Lakefront Retreat offers a tranquil setting wi...
6,Hillside Hideaway,"$600,000",3,2.0,"1,800 sqft","Nestled in the hills, this 3-bedroom, 2-bathro...",Hillside Hideaway is a secluded neighborhood w...
7,Garden Grove,"$850,000",4,3.0,"2,400 sqft","Welcome to Garden Grove, where this 4-bedroom,...",Garden Grove is a garden lover's paradise with...
8,Oceanfront Oasis,"$2,000,000",5,4.5,"4,000 sqft",Live the beachfront lifestyle in this luxuriou...,Oceanfront Oasis offers a coastal living exper...
9,Historic District Charm,"$700,000",3,2.0,"2,000 sqft",Step back in time with this charming 3-bedroom...,The Historic District is a quaint neighborhood...


## Step 3: Storing Listings in a Vector Database

In [13]:
df['Combined Description'] = df['Description'] + ' ' + df['Neighborhood Description']


In [14]:
df.head()

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description,Combined Description
0,Willow Creek,"$650,000",4,3.0,"2,500 sqft",Located in the peaceful neighborhood of Willow...,Willow Creek offers a serene setting with tree...,Located in the peaceful neighborhood of Willow...
1,Sunset Hills,"$1,200,000",5,4.0,"3,800 sqft","This luxurious 5-bedroom, 4-bathroom home in S...",Sunset Hills is known for its upscale homes an...,"This luxurious 5-bedroom, 4-bathroom home in S..."
2,Riverfront Estates,"$900,000",3,2.5,"2,300 sqft","Welcome to Riverfront Estates, where this 3-be...",Riverfront Estates is a waterfront community w...,"Welcome to Riverfront Estates, where this 3-be..."
3,Downtown Loft District,"$500,000",2,2.0,"1,200 sqft",Live in the heart of the city in this modern 2...,The Downtown Loft District is a vibrant neighb...,Live in the heart of the city in this modern 2...
4,Mountain View Terrace,"$750,000",3,2.5,"2,100 sqft",Enjoy panoramic mountain views from this 3-bed...,Mountain View Terrace offers a peaceful settin...,Enjoy panoramic mountain views from this 3-bed...


In [15]:
# Define the filename for your CSV
df.to_csv('real_estate_listings_full_description.csv')

In [16]:
chromadb.__version__

'0.5.3'

In [17]:
persist_directory = './data/vectordb'
client = chromadb.PersistentClient(path=persist_directory)
collection = client.get_or_create_collection(name="home_match", embedding_function=embedding_functions.DefaultEmbeddingFunction())
data = pd.read_csv('real_estate_listings_full_description.csv')
descriptions = data['Combined Description'].tolist()

# Load a pre-trained embedding model
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to generate embeddings
def generate_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.numpy()

# Generate embeddings for the descriptions
embeddings = generate_embeddings(descriptions)
embeddings = [embedding.tolist() for embedding in embeddings]
for idx, row in data.iterrows():
    collection.add(
        documents=[row['Combined Description']],
        metadatas=[{
            'Neighborhood': row['Neighborhood'],
            'Price': row['Price'],
            'Bedrooms': row['Bedrooms'],
            'Bathrooms': row['Bathrooms'],
            'House Size': row['House Size'],
            'Description': row['Description'],
            'Neighborhood Description': row['Neighborhood Description']
        }],
        embeddings=[embeddings[idx]],
        ids=[str(idx)]
    )

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of e

In [18]:
collection.peek()

{'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'embeddings': [[0.247714102268219,
   -0.23775500059127808,
   0.14041803777217865,
   -0.04487494379281998,
   -0.07222452759742737,
   0.04240354150533676,
   0.13279767334461212,
   -0.12043432146310806,
   -0.08178139477968216,
   0.115142822265625,
   -0.0028462938498705626,
   0.04469313099980354,
   0.08328341692686081,
   0.018352672457695007,
   0.1642753779888153,
   0.03243514150381088,
   0.03837430849671364,
   -0.04509450122714043,
   0.09986776858568192,
   -0.09505770355463028,
   0.05794065445661545,
   0.14007322490215302,
   0.20151318609714508,
   0.15073341131210327,
   -0.023656168952584267,
   0.10360842198133469,
   -0.10708685219287872,
   0.10530029237270355,
   -0.10340721160173416,
   -0.04413963481783867,
   0.1299566924571991,
   0.03496536612510681,
   -0.127603217959404,
   -0.14135165512561798,
   -0.0016919492045417428,
   -0.07802751660346985,
   -0.1003846675157547,
   -0.13106919825077057,

## Step 4: Building the User Preference Interface

In [19]:
def collect_user_inputs(questions):
    preferences = {}
    for question in questions:
        response = input(question + " ")
        preferences[question] = response
    return preferences

questions = [
    "How big do you want your house to be?", 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "How urban do you want your neighborhood to be?"
]

buyer_preferences = collect_user_inputs(questions)

How big do you want your house to be? I want my house to be at least 2500sq and have 4 bedrooms
What are 3 most important things for you in choosing this property? I want to be in a good school district, have spacious rooms and be in a family friendly neighborhood
Which amenities would you like? I would like to be in a very outdoorsy type neighborhood with access to trails, modern architecture, and scenic views
How urban do you want your neighborhood to be? I would like to be in the suburbs but still close to the city.


In [20]:
buyer_preferences

{'How big do you want your house to be?': 'I want my house to be at least 2500sq and have 4 bedrooms',
 'What are 3 most important things for you in choosing this property?': 'I want to be in a good school district, have spacious rooms and be in a family friendly neighborhood',
 'Which amenities would you like?': 'I would like to be in a very outdoorsy type neighborhood with access to trails, modern architecture, and scenic views',
 'How urban do you want your neighborhood to be?': 'I would like to be in the suburbs but still close to the city.'}

## Step 5: Searching Based on Preferences

In [21]:
def semantic_search(collection, preferences):
    # Combine preferences into a single search query
    search_query = " ".join(preferences.values())
    
    # Perform semantic search
    results = collection.query(
        query_texts=[search_query],
        n_results=3  # Retrieve top 3 results
    )
    
    return results

# Perform the search
search_results = semantic_search(collection, buyer_preferences)

/home/student/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 61.1MiB/s]


In [22]:
search_results

{'ids': [['1', '9', '0']],
 'distances': [[3.69081016468799, 3.8761468340038374, 4.835502272500808]],
 'metadatas': [[{'Bathrooms': 4,
    'Bedrooms': 5,
    'Description': 'Located in the prestigious Sunset Hills neighborhood, this stunning 5-bedroom, 4-bathroom home offers luxury living at its finest. The grand foyer leads to a gourmet kitchen, formal dining room, and spacious living area with panoramic views. The master suite features a spa-like bathroom and walk-in closet. The backyard retreat includes a pool, spa, and outdoor fireplace.',
    'House Size': '3,200 sqft',
    'Neighborhood': 'Sunset Hills',
    'Neighborhood Description': 'Sunset Hills is known for its upscale homes, tree-lined streets, and exclusive country club. Residents enjoy access to top-rated schools, golf courses, and fine dining options in this sought-after community.',
    'Price': '$900,000'},
   {'Bathrooms': 3,
    'Bedrooms': 4,
    'Description': 'Discover the beauty of Valley View Heights in this 4-b

## Step 6: Personalizing Listing Descriptions
### I developed the prompt below through testing it several times and seeing the response given from the LLM

In [28]:
model_name = 'gpt-4o'
llm = OpenAI(model_name=model_name, temperature=0, max_tokens=4000)
prompt_template = f"""
You are a realtor working for Amogh's Smart Property Solutions, providing information in a presentation to a prospective buyer about several houses your AI algorithm has picked out based on their preferences. Augment the descriptions for the top properties and provide a detailed story about each property to appeal to the customers preferences and including all the factual data about the properties especially price. Start the presentation by describing how these properties were tailored just for the client. In the response, dont include any asterisks or hashtags. For every of the properties description just have one section per property.
The buyer has the following preferences:
- House Size: {buyer_preferences['How big do you want your house to be?']}
- Important Factors: {buyer_preferences['What are 3 most important things for you in choosing this property?']}
- Amenities: {buyer_preferences['Which amenities would you like?']}
- Neighborhood: {buyer_preferences['How urban do you want your neighborhood to be?']}

Based on these preferences, here are three properties that match their criteria:

1. {search_results['metadatas'][0][0]['Neighborhood']}
   - Description: {search_results['metadatas'][0][0]['Description']}
   - Size: {search_results['metadatas'][0][0]['House Size']}
   - Bedrooms/Bathrooms: {search_results['metadatas'][0][0]['Bedrooms']} beds / {search_results['metadatas'][0][0]['Bathrooms']} baths
   - Price: {search_results['metadatas'][0][0]['Price']}
   - Neighborhood: {search_results['metadatas'][0][0]['Neighborhood']}
   - Neighborhood Description: {search_results['metadatas'][0][0]['Neighborhood Description']}
   
2. {search_results['metadatas'][0][1]['Neighborhood']}
   - Description: {search_results['metadatas'][0][1]['Description']}
   - Size: {search_results['metadatas'][0][1]['House Size']}
   - Bedrooms/Bathrooms: {search_results['metadatas'][0][1]['Bedrooms']} beds / {search_results['metadatas'][0][1]['Bathrooms']} baths
   - Price: {search_results['metadatas'][0][1]['Price']}
   - Neighborhood: {search_results['metadatas'][0][1]['Neighborhood']}
   - Neighborhood Description: {search_results['metadatas'][0][1]['Neighborhood Description']}
   
3. {search_results['metadatas'][0][2]['Neighborhood']}
   - Description: {search_results['metadatas'][0][2]['Description']}
   - Size: {search_results['metadatas'][0][2]['House Size']}
   - Bedrooms/Bathrooms: {search_results['metadatas'][0][2]['Bedrooms']} beds / {search_results['metadatas'][0][2]['Bathrooms']} baths
   - Price: {search_results['metadatas'][0][2]['Price']}
   - Neighborhood: {search_results['metadatas'][0][2]['Neighborhood']}
   - Neighborhood Description: {search_results['metadatas'][0][2]['Neighborhood Description']}
"""
prompt_template

"\nYou are a realtor working for Amogh's Smart Property Solutions, providing information in a presentation to a prospective buyer about several houses your AI algorithm has picked out based on their preferences. Augment the descriptions for the top properties and provide a detailed story about each property to appeal to the customers preferences and including all the factual data about the properties especially price. Start the presentation by describing how these properties were tailored just for the client. In the response, dont include any asterisks or hashtags. For every of the properties description just have one section per property.\nThe buyer has the following preferences:\n- House Size: I want my house to be at least 2500sq and have 4 bedrooms\n- Important Factors: I want to be in a good school district, have spacious rooms and be in a family friendly neighborhood\n- Amenities: I would like to be in a very outdoorsy type neighborhood with access to trails, modern architecture,

In [29]:
response = llm(prompt=prompt_template)
print(response)

Welcome to Amogh's Smart Property Solutions! We are thrilled to present you with a selection of homes that have been meticulously tailored to your preferences. Our advanced AI algorithm has carefully analyzed your requirements and identified these top properties that perfectly align with your vision of an ideal home. Let's dive into the details of each property and explore how they meet your needs.

Sunset Hills

Located in the prestigious Sunset Hills neighborhood, this stunning 5-bedroom, 4-bathroom home offers luxury living at its finest. As you step into the grand foyer, you'll be greeted by an elegant and spacious interior that seamlessly blends modern architecture with timeless design. The gourmet kitchen is a chef's dream, featuring high-end appliances, granite countertops, and a large island perfect for family gatherings. The formal dining room and spacious living area with panoramic views create an inviting atmosphere for entertaining guests.

The master suite is a true retrea